In [1]:
import os
os.getcwd()

'C:\\Users\\kalya\\Python'

In [2]:
# !pip install spacy
!python -m spacy download en

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\kalya\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\kalya\Anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


When you `spacy download en`, spaCy tries to find the best small model that matches your spaCy distribution. The small model that I am talking about defaults to `en_core_web_sm` which can be found in different variations which correspond to the different spaCy versions (for example spacy, spacy-nightly have `en_core_web_sm` of different sizes).

When spaCy finds the best model for you, it downloads it and then links the name `en` to the package it downloaded, e.g. `en_core_web_sm`. That basically means that whenever you refer to en you will be referring to `en_core_web_sm`. In other words, en after linking is not a "real" package, is just a name for `en_core_web_sm`.

However, it doesn't work the other way. You can't refer directly to `en_core_web_sm` because your system doesn't know you have it installed. When you did `spacy download en` you basically did a pip install. So pip knows that you have a package named `en` installed for your python distribution, but knows nothing about the package `en_core_web_sm`. This package is just replacing package `en` when you import it, which means that package `en` is just a softlink to `en_core_web_sm`.

Of course, you can directly download `en_core_web_sm`, using the command: `python -m spacy download en_core_web_sm`, or you can even link the name `en` to other models as well. For example, you could do `python -m spacy download en_core_web_lg` and then `python -m spacy link en_core_web_lg en`. That would make en a name for `en_core_web_lg`, which is a large spaCy model for the English language

`python -m spacy download en_core_web_lg` or  `python -m spacy download en_core_web_sm`

`python -m spacy download en`

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
# nlp = spacy.load('en')

In [4]:
print("Enter two space-separated words") 
words = input()  
tokens = nlp(words)

for token in tokens: 
    # Printing the following attributes of each token. 
    # text: the word string
    # has_vector: if it contains a vector representation in the model
    # vector_norm: the algebraic norm of the vector
    # is_oov: if the word is out of vocabulary
    print(token.text, token.has_vector, token.vector_norm, token.is_oov, end= '||') 

token1, token2 = tokens[0], tokens[1] 
print("Similarity:", token1.similarity(token2))

Enter two space-separated words
off-of
off True 18.169401 True||- True 18.69875 True||of True 19.728752 True||Similarity: 0.089341864


C:\Users\kalya\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  


In [5]:
import numpy as np 
from spacy.vocab import Vocab 
  
nlp = spacy.load('en_core_web_sm') 
new_word = 'bucrest'
  
print('Before custom setting') 
print(vocab.get_vector('bucrest')) 
  
custom_vector = np.random.uniform(-1, 1, (300, )) 
  
vocab.set_vector(new_word, custom_vector) 
  
print('After custom setting') 
print(vocab.get_vector('bucrest'))

Before custom setting


NameError: name 'vocab' is not defined

**Topic modeling using latent semantic analysis**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD

C:\Users\kalya\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
# 20news group data set
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [17]:
print(dataset.target_names, end='')
documents[0]

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

- We will try to clean our text data as much as possible
- The idea is to remove the punctuations, numbers, and special characters all in one step using the regex replace("[^a-zA-Z#]", ""), which will replace everything, except alphabets with space
- Then we will remove shorter words because they usually don’t contain useful information
- Finally, we will make all the text lowercase to nullify case sensitivity

In [18]:
# Cleaning the data
news_df = pd.DataFrame({'document':documents})

# Removing everything except alphabets
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# Removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# Make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

news_df['clean_doc'][0]

'well sure about story seem biased what disagree with your statement that media ruin israels reputation that rediculous media most israeli media world having lived europe realize that incidences such described letter have occured media whole seem ignore them subsidizing israels existance europeans least same degree think that might reason they report more clearly atrocities what shame that austria daily reports inhuman acts commited israeli soldiers blessing received from government makes some holocaust guilt away after look jews treating other races when they power unfortunate'

- It’s good practice to remove the stop-words from the text data as they are mostly clutter and hardly carry any information - - Stop-words are terms like ‘it’, ‘they’, ‘am’, ‘been’, ‘about’, ‘because’, ‘while’, etc
- To remove stop-words from the documents, we will have to tokenize the text, i.e., split the string of text into individual tokens or words
- We will stitch the tokens back together once we have removed the stop-words

In [19]:
# Further cleaning steps
stop_words = stopwords.words('english')

# Tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# Remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# De-tokenization
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

news_df['clean_doc'][0]

'well sure story seem biased disagree statement media ruin israels reputation rediculous media israeli media world lived europe realize incidences described letter occured media whole seem ignore subsidizing israels existance europeans least degree think might reason report clearly atrocities shame austria daily reports inhuman acts commited israeli soldiers blessing received government makes holocaust guilt away look jews treating races power unfortunate'

We could have used all the terms to create this matrix but that would need quite a lot of computation time and resources. Hence, we have restricted the number of features to 1,000

In [11]:
# Document-Term Matrix
"We will use sklearn’s TfidfVectorizer to create a document-term matrix with 1,000 terms"

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

- The next step is to represent each and every term and document as a vector
- We will use the document-term matrix and decompose it into multiple matrices
- We will use sklearn’s TruncatedSVD to perform the task of matrix decomposition

Since the data comes from 20 different newsgroups, let’s try to have 20 topics for our text data. The number of topics can be specified by using the n_components parameter.

In [12]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)
len(svd_model.components_)

20

In [27]:
# The components of svd_model are our topics, and we can access them using svd_model.components_. Finally, let’s print a
# few most important words in each of the 20 topics and see how our model has done

terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("\nTopic_"+str(i)+": ")
    for t in sorted_terms:
        print(t[0], end=' ')


Topic_0: 
like know people think good time thanks 
Topic_1: 
thanks windows card drive mail file advance 
Topic_2: 
game team year games season players good 
Topic_3: 
drive scsi disk hard card drives problem 
Topic_4: 
windows file window files program using problem 
Topic_5: 
government chip mail space information encryption data 
Topic_6: 
like bike know chip sounds looks look 
Topic_7: 
card sale video offer monitor price jesus 
Topic_8: 
know card chip video government people clipper 
Topic_9: 
good know time bike jesus problem work 
Topic_10: 
think chip good thanks clipper need encryption 
Topic_11: 
thanks right problem good bike time window 
Topic_12: 
good people windows know file sale files 
Topic_13: 
space think know nasa problem year israel 
Topic_14: 
space good card people time nasa thanks 
Topic_15: 
people problem window time game want bike 
Topic_16: 
time bike right windows file need really 
Topic_17: 
time problem file think israel long mail 
Topic_18: 
file need 